# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [157]:

## make sure the following dependencies are installed, specifically geopy
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import requests
import scipy.stats
import random as rd
from citipy import citipy


## Generate Cities List

In [158]:
#### having isssues with this code. I get a lot of duplicate values for rand_lat_lon_df.at[x,'Exact Coor']

## generates 2 lists of random floats for latitude and longitude
### maybe I could generate a larger list and narrow it down by population size(whats my max, 5000????)
samp_size = 2000
lat_list = [round(rd.uniform(-90,90),2) for num in range(samp_size)]
lon_list = [round(rd.uniform(-180,180),2) for num in range(samp_size)]

city_name_list=[]
country_code_list=[]
ex_coor_list=[]

new_world_keys=list(citipy.WORLD_CITIES_DICT.keys())
new_world_values=list(citipy.WORLD_CITIES_DICT.values())

for y in range(samp_size):
    ap_lat=lat_list[y]
    ap_lon=lon_list[y]
    current_city=citipy.nearest_city(ap_lat,ap_lon)
    city_name_list.append(current_city.city_name)
    country_code_list.append(current_city.country_code)
    nwvi=new_world_values.index(current_city)
    ex_coor_list.append(new_world_keys[nwvi])
 
coor_df= pd.DataFrame({
    'Approximate Lat':lat_list,
    'Approximate Lon':lon_list,
    'City Name':city_name_list,
    'Country Code': country_code_list,
    'Exact Coordinates':ex_coor_list
})

print(len(coor_df['Approximate Lat']))
coor_df=coor_df.drop_duplicates(subset=['Exact Coordinates'],keep='first')
print(len(coor_df['Approximate Lat']))

coor_df.head()






2000
770


,Approximate Lat,Approximate Lon,City Name,Country Code,Exact Coordinates
0,15.20,148.13,airai,pw,"(7.3575, 134.5577778)"
1,-53.03,31.13,port alfred,za,"(-33.590572, 26.891037)"
2,50.34,47.68,piterka,ru,"(50.680297, 47.445395)"
3,88.70,-11.44,illoqqortoormiut,gl,"(70.4833333, -21.9666667)"
4,13.26,-12.82,youkounkoun,gn,"(12.5333333, -13.1333333)"


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [160]:
#  Get the indices of cities that have humidity over 100%.


In [161]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression